In [173]:
import pandas as pd
import numpy as np

In [174]:
# Drop duplicate elements
data = pd.read_csv('../Data/data_original.csv').copy()
data = data.drop_duplicates()
data = data[data['Tỉnh/Thành phố'].str.contains('Hà Nội', case=False, na=False)].reset_index(drop=True)
data = data[~data['Mức giá'].str.contains('Thỏa thuận', case=False, na=False)].reset_index(drop=True)
data = data[~data['Số phòng ngủ'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Số toilet'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Pháp lý'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Nội thất'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
# data = data[~data['Mặt tiền'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
# data = data[~data['Hướng nhà'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
# data = data[~data['Hướng ban công'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data

,Xã/Phường,Quận/Huyện,Tỉnh/Thành phố,Chủ đầu tư,Tên dự án,Diện tích,Mức giá,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Mặt tiền,Hướng nhà,Hướng ban công,Thông tin khác,Lịch sử giá,Khoảng giá
0,Phường Quảng An,Tây Hồ,Hà Nội,Tân Hoàng Minh Group,D’. Le Roi Soleil,146 m²,"13,76 tỷ",3 phòng,2 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ,Không có thông tin,Không có thông tin,Không có thông tin,Không có thông tin,"Giá T10/22: 75,7 tr/m²; Giá T11/22: 79,6 tr/m²...","62,5 104,2; 65,6 98,9; 64,4 97,6; 60 108,9; 68..."
1,Phường Hoàng Văn Thụ,Hoàng Mai,Hà Nội,Tập đoàn Lã Vọng,Louis City Hoàng Mai,95 m²,"27,5 tỷ",6 phòng,6 phòng,Sổ đỏ/ Sổ hồng,Không nội thất,5 m,Đông,Đông,"Đường vào: 17,5 m; Số tầng: 5 tầng",Giá T10/22: 150 tr/m²; Giá T11/22: 150 tr/m²; ...,"131,6 182,7; 130 197; 128,5 198; 133 256,6; 13..."
2,Phường Hoàng Văn Thụ,Hoàng Mai,Hà Nội,Tập đoàn Lã Vọng,Louis City Hoàng Mai,"112,8 m²",38 tỷ,6 phòng,6 phòng,Sổ đỏ/ Sổ hồng.,Không nội thất.,6 m,Tây,Tây,Đường vào: 40 m; Số tầng: 5 tầng,Giá T10/22: 150 tr/m²; Giá T11/22: 150 tr/m²; ...,"131,6 182,7; 130 197; 128,5 198; 133 256,6; 13..."
3,Phường Hoàng Văn Thụ,Hoàng Mai,Hà Nội,Tập đoàn Lã Vọng,Louis City Hoàng Mai,113 m²,270 triệu/m²,5 phòng,6 phòng,Sổ đỏ/ Sổ hồng.,Cơ bản.,6 m,Tây,Tây,Đường vào: 40 m; Số tầng: 5 tầng,Giá T10/22: 150 tr/m²; Giá T11/22: 150 tr/m²; ...,"131,6 182,7; 130 197; 128,5 198; 133 256,6; 13..."
4,Xã Dương Xá,Gia Lâm,Hà Nội,Tập đoàn Vingroup,Vinhomes Ocean Park Gia Lâm,55 m²,"2,55 tỷ",2 phòng,1 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ,Không có thông tin,Đông - Bắc,Tây - Nam,Không có thông tin,"Giá T10/22: 31,6 tr/m²; Giá T11/22: 30,6 tr/m²...","27,4 45; 27,2 41,4; 26,5 38,3; 25,4 42,5; 25,7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,Phường Tây Mỗ,Nam Từ Liêm,Hà Nội,Masterise Homes,Masteri West Heights,"62,3 m²","4,6 tỷ",2 phòng,2 phòng,Hợp đồng mua bán,Đầy đủ,Không có thông tin,Tây - Nam,Đông - Bắc,Không có thông tin,Không có dữ liệu lịch sử giá,Không có dữ liệu lịch sử khoảng giá
465,Phường Tây Mỗ,Nam Từ Liêm,Hà Nội,Tập đoàn Vingroup,The Miami,"42,8 m²","2,55 tỷ",1 phòng,1 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ,Không có thông tin,Tây - Bắc,Đông - Nam,Không có thông tin,Không có dữ liệu lịch sử giá,Không có dữ liệu lịch sử khoảng giá
466,Phường Tây Mỗ,Nam Từ Liêm,Hà Nội,Tập đoàn Vingroup,The Tonkin - Vinhomes Smart City,"72,1 m²","4,65 tỷ",2 phòng,2 phòng,Sổ đỏ/ Sổ hồng,Cơ bản,Không có thông tin,Đông - Bắc,Tây - Nam,Không có thông tin,Không có dữ liệu lịch sử giá,Không có dữ liệu lịch sử khoảng giá
467,Xã Di Trạch,Hoài Đức,Hà Nội,Công ty Vietracimex,Hinode Royal Park,311 m²,40 tỷ,6 phòng,5 phòng,Hợp đồng mua bán,Không nội thất,12 m,Đông - Bắc,Đông - Bắc,Đường vào: 17 m; Số tầng: 3 tầng,Không có dữ liệu lịch sử giá,Không có dữ liệu lịch sử khoảng giá


In [175]:
# Clean area Data
area = data['Diện tích']
cleaned_area = area.str.replace(' m²', '').str.replace('.', '').str.replace(',', '.').astype(np.float32)
cleaned_area

0      146.000000
1       95.000000
2      112.800003
3      113.000000
4       55.000000
          ...    
464     62.299999
465     42.799999
466     72.099998
467    311.000000
468     64.500000
Name: Diện tích, Length: 469, dtype: float32

In [176]:
# Clean price Data

price = data['Mức giá']
# Convert to price per square
price_area = price.str.contains('tỷ', case=False, na=False)
cleaned_price_area = data.loc[price_area, 'Mức giá'].str.replace(' tỷ', '').str.replace(',', '.').astype(np.float32)*1000
price_per_square = (cleaned_price_area / cleaned_area[price_area])
data.loc[price_area, 'Mức giá'] = price_per_square

# Clean form
wrong_form = price.str.contains(' triệu/m²', case=False, na=False)
cleaned_wrong_form = data.loc[wrong_form, 'Mức giá'].str.replace(' triệu/m²', '').str.replace(',', '.').astype(np.float32)
data.loc[wrong_form, 'Mức giá'] = cleaned_wrong_form

price


0       94.246574
1      289.473694
2      336.879425
3           270.0
4       46.363636
          ...    
464     73.836273
465     59.579441
466     64.493759
467    128.617371
468     60.465115
Name: Mức giá, Length: 469, dtype: object

In [177]:
# Clean room

bed_room = data['Số phòng ngủ']
cleaned_bed_room = bed_room.str.replace(' phòng', '').astype(np.float32)

toilet_room = data['Số toilet']
cleaned_toilet_room = toilet_room.str.replace(' phòng', '').astype(np.float32)


In [178]:
# Clean legal

legal = data['Pháp lý']
wrong_form = legal.str.contains('HĐMB', case=False, na=False)
data.loc[wrong_form, 'Pháp lý'] = data.loc[wrong_form, 'Pháp lý'].str.replace('HĐMB', "Hợp đồng mua bán")

legal


0        Sổ đỏ/ Sổ hồng
1        Sổ đỏ/ Sổ hồng
2       Sổ đỏ/ Sổ hồng.
3       Sổ đỏ/ Sổ hồng.
4        Sổ đỏ/ Sổ hồng
             ...       
464    Hợp đồng mua bán
465      Sổ đỏ/ Sổ hồng
466      Sổ đỏ/ Sổ hồng
467    Hợp đồng mua bán
468    Hợp đồng mua bán
Name: Pháp lý, Length: 469, dtype: object

In [179]:
# Clean front

front = data['Mặt tiền']
wrong_form = front.str.contains(' m', case=False, na=False)
data.loc[wrong_form, 'Mặt tiền'] = data.loc[wrong_form, 'Mặt tiền'].str.replace(' m', '').str.replace(',', '.').astype(np.float32)
front

0      Không có thông tin
1                     5.0
2                     6.0
3                     6.0
4      Không có thông tin
              ...        
464    Không có thông tin
465    Không có thông tin
466    Không có thông tin
467                  12.0
468    Không có thông tin
Name: Mặt tiền, Length: 469, dtype: object

In [180]:
cleaned_data = pd.DataFrame({
    'Xã/Phường': data['Xã/Phường'],
    'Quận/Huyện': data['Quận/Huyện'],
    'Tỉnh/Thành phố': data['Tỉnh/Thành phố'],
    'Chủ đầu tư': data['Chủ đầu tư'],
    'Tên dự án': data['Tên dự án'],
    'Diện tích': area,
    'Mức giá': price,
    'Số phòng ngủ': bed_room,
    'Số toilet': toilet_room,
    'Pháp lý': legal,
    'Nội thất': data['Nội thất'],
    'Mặt tiền': front,
    'Hướng nhà': data['Hướng nhà'],
    'Thông tin khác': data['Thông tin khác'],
    'Lịch sử giá': data['Lịch sử giá'],
    'Khoảng giá': data['Khoảng giá']
})

# Lưu DataFrame vào file CSV
cleaned_data.to_csv('../Data/cleaned_data.csv', index=False)

# In kết quả để kiểm tra
print(cleaned_data)

                Xã/Phường   Quận/Huyện Tỉnh/Thành phố            Chủ đầu tư  \
0         Phường Quảng An       Tây Hồ         Hà Nội  Tân Hoàng Minh Group   
1    Phường Hoàng Văn Thụ    Hoàng Mai         Hà Nội      Tập đoàn Lã Vọng   
2    Phường Hoàng Văn Thụ    Hoàng Mai         Hà Nội      Tập đoàn Lã Vọng   
3    Phường Hoàng Văn Thụ    Hoàng Mai         Hà Nội      Tập đoàn Lã Vọng   
4             Xã Dương Xá      Gia Lâm         Hà Nội     Tập đoàn Vingroup   
..                    ...          ...            ...                   ...   
464         Phường Tây Mỗ  Nam Từ Liêm         Hà Nội       Masterise Homes   
465         Phường Tây Mỗ  Nam Từ Liêm         Hà Nội     Tập đoàn Vingroup   
466         Phường Tây Mỗ  Nam Từ Liêm         Hà Nội     Tập đoàn Vingroup   
467           Xã Di Trạch     Hoài Đức         Hà Nội   Công ty Vietracimex   
468         Phường Tây Mỗ  Nam Từ Liêm        Hà Nội.     Tập đoàn Vingroup   

                            Tên dự án Diện tích    